In [1]:
from opxrd.wrapper import OpXRD

# ------------------------------------------------------------

smoltest_dirpath = '/home/daniel/aimat/data/opXRD/test_smol'
bigtest_dirpath = '/home/daniel/aimat/data/opXRD/test'
test_databases = OpXRD.load_project_list(root_dirpath=smoltest_dirpath)

- Loading databases from /home/daniel/aimat/data/opXRD/test_smol
[2024-12-20 09:31:31]: Loading patterns from local dirpath /home/daniel/aimat/data/opXRD/test_smol/USC


100% (174 of 174) |######################| Elapsed Time: 0:00:00 Time:  0:00:000:00


[2024-12-20 09:31:32]: Finished loading pattern database located at /home/daniel/aimat/data/opXRD/test_smol/USC
[2024-12-20 09:31:32]: Successfully extracted 174 patterns from 174/174 xrd files
[2024-12-20 09:31:32]: Loading patterns from local dirpath /home/daniel/aimat/data/opXRD/test_smol/CNRS


100% (210 of 210) |######################| Elapsed Time: 0:00:01 Time:  0:00:010000


[2024-12-20 09:31:33]: Finished loading pattern database located at /home/daniel/aimat/data/opXRD/test_smol/CNRS
[2024-12-20 09:31:33]: Successfully extracted 210 patterns from 210/210 xrd files


In [2]:
from opxrd.analysis import DatabaseAnalyser

analyser = DatabaseAnalyser(databases=test_databases, output_dirpath='/tmp/opxrd_analysis')

In [3]:
analyser.plot_fourier_reference(b=5, c= 0.4)

---> Fourier transform of a Gaussian function $I(x) = e^{{-0.5(x-b)^2/c}$$b = 5, c =  0.4$

TypeError: 'numpy.ndarray' object is not callable

In [ ]:
analyser.plot_fourier()

In [ ]:
analyser.plot_in_single(limit_patterns=10)

In [ ]:
analyser.plot_in_single(limit_patterns=50)

In [ ]:
analyser.plot_in_single(limit_patterns=1000)

In [ ]:
analyser.plot_effective_components()

In [ ]:
analyser.plot_histogram()

In [ ]:
analyser.show_label_fractions()
analyser.print_total_counts()